# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import holoviews as hv
hv.extension('bokeh')
import hvplot.pandas
import hvplot
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

Output hidden; open in https://colab.research.google.com to view.

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,acarau,-2.8856,-40.1200,85.17,69,90,9.62,BR,1684420297
1,1,puerto natales,-51.7236,-72.4875,37.85,87,40,17.27,CL,1684420297
2,2,keflavik,64.0049,-22.5624,51.71,100,100,20.71,IS,1684420298
3,3,whitehorse,60.7161,-135.0538,43.57,52,0,5.75,CA,1684420298
4,4,punta arenas,-53.1500,-70.9167,39.31,87,75,18.41,CL,1684420298


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display


# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_condition_df = city_data_df.loc[(city_data_df["Max Temp"] >= 10) 
                                & (city_data_df["Max Temp"] <= 90)
                                & (city_data_df["Humidity"] <= 80)
                                & (city_data_df["Cloudiness"] <= 30), :]


# Drop any rows with null values
ideal_weather_condition_df = ideal_weather_condition_df.dropna()

# Display sample data
ideal_weather_condition_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,whitehorse,60.7161,-135.0538,43.57,52,0,5.75,CA,1684420298
7,7,pahrump,36.2083,-115.9839,77.45,28,4,4.00,US,1684420299
9,9,jamestown,42.0970,-79.2353,53.02,43,0,5.75,US,1684420179
10,10,longyearbyen,78.2186,15.6401,26.44,59,0,4.61,SJ,1684420299
13,13,magnolia,33.2671,-93.2393,74.88,69,2,3.89,US,1684420300


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_condition_df[['City','Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,whitehorse,CA,60.7161,-135.0538,52,
7,pahrump,US,36.2083,-115.9839,28,
9,jamestown,US,42.0970,-79.2353,43,
10,longyearbyen,SJ,78.2186,15.6401,59,
13,magnolia,US,33.2671,-93.2393,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 100,
    "filter": "",
    "bias": "",
    "apiKey": geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = (f"circle:{longitude},{latitude},{radius}")
    params["bias"] = (f"proximity:{longitude},{latitude}")
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
whitehorse - nearest hotel: Town & Mountain Hotel
pahrump - nearest hotel: Holiday Inn Express & Suites
jamestown - nearest hotel: DoubleTree Jamestown
longyearbyen - nearest hotel: The Vault
magnolia - nearest hotel: Holiday Inn Express & Suites
georgetown - nearest hotel: Page 63 hostel
salym - nearest hotel: Каравани
kas - nearest hotel: KEKOVA OTEL
albany - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
cambria - nearest hotel: Olallieberry Inn
kruisfontein - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
uturoa - nearest hotel: Hawaiki Nui hotel
swan hill - nearest hotel: Murray Downs Golf Resort
campoalegre - nearest hotel: No hotel found
ketchikan - nearest hotel: Cape Fox Lodge
dalandzadgad - nearest hotel: Хан Уул буудал
karratha - nearest hotel: Karratha International Hotel
yinchuan - nearest hotel: Yinchuan hotel
copala - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
west bay - near

,City,Country,Lat,Lng,Humidity,Hotel Name
3,whitehorse,CA,60.7161,-135.0538,52,Town & Mountain Hotel
7,pahrump,US,36.2083,-115.9839,28,Holiday Inn Express & Suites
9,jamestown,US,42.0970,-79.2353,43,DoubleTree Jamestown
10,longyearbyen,SJ,78.2186,15.6401,59,The Vault
13,magnolia,US,33.2671,-93.2393,69,Holiday Inn Express & Suites
...,...,...,...,...,...,...
404,dickson city,US,41.4715,-75.6077,36,Days Inn by Wyndham Scranton PA
405,fort bragg,US,35.1390,-79.0060,59,Airborne Inn Lodging
407,galeana,MX,24.8333,-100.0667,51,Hotel Jardin
412,verkhnevilyuysk,RU,63.4453,120.3167,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot
